In [6]:
#2.1
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Map,Timeline,Pie,Page,Line
tr = Timeline().add_schema(play_interval=1000, height=40, is_loop_play=True,is_auto_play=True,control_position = "left",is_rewind_play= False)#时间轴
tl = Timeline().add_schema(play_interval=1000, height=40, is_loop_play=True,is_auto_play=True,control_position = "left",is_rewind_play= False)#时间轴
list1 = []#存放日期
list2 = []#存放确诊人数
list3 = []#存放治愈人数
list4 = []#存放死亡人数
task2 = pd.read_csv('task1_2.csv',encoding='utf-8')
for i in range(1,7):#以月为单位进行遍历 得到1月到i月各个省份的累计确诊，治愈，死亡人数
    list1.append("2020-0{}".format(i))#存入日期
    data2 = task2 #防止改变data
    data2['日期'] = pd.to_datetime(data2['日期'])#转换日期类型
    data2 = data2.set_index('日期')#将日期作为索引
    data2 = data2.sort_index()#对日期进行排序 以便于对其进行切分
    if i%2 == 1:#1，3，5月
        data2 = data2.truncate('2020-01','2020-0{}-31'.format(i))#对数据进行切分，得到1月到i月的全部数据
    else:
        if i == 2:#2月
            data2 = data2.truncate('2020-01','2020-0{}-28'.format(i))#对数据进行切分，得到1月到i月的全部数据
        else:#4，6月
            data2 = data2.truncate('2020-01','2020-0{}-30'.format(i))#对数据进行切分，得到1月到i月的全部数据
    data2 = data2.groupby('省份', as_index=False).sum()#对1月到i月的各个省份的各项数据进行合并 此时新增确诊人数即为累计确诊人数
    data2['日期'] = '2020-0{}'.format(i)#加入日期列
    list2.append(int(data2.groupby('日期').sum()['新增确诊']))#存入确诊人数
    list3.append(int(data2.groupby('日期').sum()['新增治愈']))#存入治愈人数
    list4.append(int(data2.groupby('日期').sum()['新增死亡']))#存入死亡人数

for i in range(1,7):#以月为单位进行遍历
    data1 = task2#防止改变data
    data1['日期'] = pd.to_datetime(data1['日期'])#转换日期类型
    data1 = data1.set_index('日期')#将日期作为索引
    data1 = data1.sort_index()#对日期进行排序 以便于对其进行切分
    if i%2 == 1:
        data1 = data1.truncate('2020-01','2020-0{}-31'.format(i))
    else:
        if i == 2:
            data1 = data1.truncate('2020-01','2020-0{}-28'.format(i))
        else:
            data1 = data1.truncate('2020-01','2020-0{}-30'.format(i))
    data1 = data1.groupby('省份', as_index=False).sum()#对1月到i月的各个省份的各项数据进行合并 此时新增确诊人数即为累计确诊人数  去除了日期列
    data1['日期'] = '2020-0{}'.format(i)#加入日期列
    data2 =data1
    data2 = data2.sort_values(by='新增确诊', ascending=False)#从大到小排序‘新增确诊’这一列，方便得到累计确诊人数前五的省份
    x = data2['省份'].tolist()#转换为list
    y = data2['新增确诊'].tolist()
    map = (#进行大屏显示 画地图
            Map()
                .add(series_name="确诊病例",  # 系列名称，用于 tooltip 的显示，legend 的图例筛选。
                     data_pair=[list(z) for z in zip(x, y)],  # 数据项 (坐标点名称，坐标点值)
                     maptype='china')  # 地图类型
                .set_global_opts(  # 全局配置项
                title_opts=opts.TitleOpts(title='疫情地图'),  # title_opts = opts.TitleOpts()主标题函数
                visualmap_opts=opts.VisualMapOpts(  # visualmap_opts=opts.VisualMapOpts()视觉映射函数
                    is_piecewise=True,  # 是否分段
                    # 自定义的每一段的范围，以及每一段的文字，以及每一段的特别的样式。
                    pieces=[{"max": 9, "min": 0, "label": "0-9", "color": "#FFECEC"},
                            {"max": 99, "min": 10, "label": "10-99", "color": "	#FFB5B5"},
                            {"max": 499, "min": 100, "label": "100-4999", "color": "#ff7575"},
                            {"max": 999, "min": 500, "label": "500-999", "color": "#FF2D2D"},
                            {"max": 4999, "min": 1000, "label": "1000-4999", "color": "	#EA0000"},
                            {"max": 9999, "min": 5000, "label": "5000-9999", "color": "#AE0000"},
                            {"max": 999999, "min": 9999, "label": ">9999", "color": "	#750000"},]
                )
            )
        )
    pie = (#画饼图
        Pie()

            .add(
            series_name="累计确诊",
            radius=["40%", "55%"],
            data_pair=[list(z) for z in zip(x[:3], y[:3])],#前3个省份
            label_opts=opts.LabelOpts(
                position="outside",
                formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
                background_color="#eee",
                border_color="#aaa",
                border_width=1,
                border_radius=4,
                rich={
                    "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                    "abg": {
                        "backgroundColor": "#e3e3e3",
                        "width": "100%",
                        "align": "right",
                        "height": 22,
                        "borderRadius": [4, 4, 0, 0],
                    },
                    "hr": {
                        "borderColor": "#aaa",
                        "width": "100%",
                        "borderWidth": 0.5,
                        "height": 0,
                    },
                    "b": {"fontSize": 16, "lineHeight": 33},
                    "per": {
                        "color": "#eee",
                        "backgroundColor": "#334455",
                        "padding": [2, 4],
                        "borderRadius": 2,
                    },
                },
            )
        )
            .set_global_opts(legend_opts=opts.LegendOpts(pos_left="left", orient="vertical"))
            .set_series_opts(
            tooltip_opts=opts.TooltipOpts(
                trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
            )
        )
    )
    line = (#画折线图
        Line()
            .add_xaxis(xaxis_data = list1)
            .add_yaxis("累计确诊人数", y_axis=list2, is_smooth=True)
            .add_yaxis("累计治愈人数", y_axis=list3, is_smooth=True)
            .add_yaxis("累计确诊死亡", y_axis=list4, is_smooth=True)
            .set_series_opts(
                areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
             )
            .set_global_opts(
            title_opts=opts.TitleOpts(title="全球疫情"),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        )
    )
    tl.add(pie, "2020-0{}".format(i))# 将日期加入到timeline中
    tr.add(map, "2020-0{}".format(i))  # 将日期加入到timeline中
page = Page(layout=Page.DraggablePageLayout)#合并在一页
page.add(tr,tl,line)
page.render('task2_1.html')#将结果保存

C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2059962679.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  list2.append(int(data2.groupby('日期').sum()['新增确诊']))#存入确诊人数
C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2059962679.py:28: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  list3.append(int(data2.groupby('日期').sum()['新增治愈']))#存入治愈人数
C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2059962679.py:29: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should 

'c:\\Users\\吕强\\Desktop\\week3\\task2_1.html'

In [7]:
name_map = {
    'Singapore Rep.': '新加坡',
    'Dominican Rep.': '多米尼加',
    'Palestine': '巴勒斯坦',
    'Bahamas': '巴哈马',
    'Timor-Leste': '东帝汶',
    'Afghanistan': '阿富汗',
    'Guinea-Bissau': '几内亚比绍',
    "Côte d'Ivoire": '科特迪瓦',
    'Siachen Glacier': '锡亚琴冰川',
    "Br. Indian Ocean Ter.": '英属印度洋领土',
    'Angola': '安哥拉',
    'Albania': '阿尔巴尼亚',
    'United Arab Emirates': '阿联酋',
    'Argentina': '阿根廷',
    'Armenia': '亚美尼亚',
    'French Southern and Antarctic Lands': '法属南半球和南极领地',
    'Australia': '澳大利亚',
    'Austria': '奥地利',
    'Azerbaijan': '阿塞拜疆',
    'Burundi': '布隆迪',
    'Belgium': '比利时',
    'Benin': '贝宁',
    'Burkina Faso': '布基纳法索',
    'Bangladesh': '孟加拉国',
    'Bulgaria': '保加利亚',
    'The Bahamas': '巴哈马',
    'Bosnia and Herz.': '波斯尼亚和黑塞哥维那',
    'Belarus': '白俄罗斯',
    'Belize': '伯利兹',
    'Bermuda': '百慕大',
    'Bolivia': '玻利维亚',
    'Brazil': '巴西',
    'Brunei': '文莱',
    'Bhutan': '不丹',
    'Botswana': '博茨瓦纳',
    'Central African Rep.': '中非',
    'Canada': '加拿大',
    'Switzerland': '瑞士',
    'Chile': '智利',
    'China': '中国',
    'Ivory Coast': '象牙海岸',
    'Cameroon': '喀麦隆',
    'Dem. Rep. Congo': '刚果民主共和国',
    'Congo': '刚果',
    'Colombia': '哥伦比亚',
    'Costa Rica': '哥斯达黎加',
    'Cuba': '古巴',
    'N. Cyprus': '北塞浦路斯',
    'Cyprus': '塞浦路斯',
    'Czech Rep.': '捷克',
    'Germany': '德国',
    'Djibouti': '吉布提',
    'Denmark': '丹麦',
    'Algeria': '阿尔及利亚',
    'Ecuador': '厄瓜多尔',
    'Egypt': '埃及',
    'Eritrea': '厄立特里亚',
    'Spain': '西班牙',
    'Estonia': '爱沙尼亚',
    'Ethiopia': '埃塞俄比亚',
    'Finland': '芬兰',
    'Fiji': '斐',
    'Falkland Islands': '福克兰群岛',
    'France': '法国',
    'Gabon': '加蓬',
    'United Kingdom': '英国',
    'Georgia': '格鲁吉亚',
    'Ghana': '加纳',
    'Guinea': '几内亚',
    'Gambia': '冈比亚',
    'Guinea Bissau': '几内亚比绍',
    'Eq. Guinea': '赤道几内亚',
    'Greece': '希腊',
    'Greenland': '格陵兰',
    'Guatemala': '危地马拉',
    'French Guiana': '法属圭亚那',
    'Guyana': '圭亚那',
    'Honduras': '洪都拉斯',
    'Croatia': '克罗地亚',
    'Haiti': '海地',
    'Hungary': '匈牙利',
    'Indonesia': '印度尼西亚',
    'India': '印度',
    'Ireland': '爱尔兰',
    'Iran': '伊朗',
    'Iraq': '伊拉克',
    'Iceland': '冰岛',
    'Israel': '以色列',
    'Italy': '意大利',
    'Jamaica': '牙买加',
    'Jordan': '约旦',
    'Japan': '日本',
    'Kazakhstan': '哈萨克斯坦',
    'Kenya': '肯尼亚',
    'Kyrgyzstan': '吉尔吉斯斯坦',
    'Cambodia': '柬埔寨',
    'Korea': '韩国',
    'Kosovo': '科索沃',
    'Kuwait': '科威特',
    'Lao PDR': '老挝',
    'Lebanon': '黎巴嫩',
    'Liberia': '利比里亚',
    'Libya': '利比亚',
    'Sri Lanka': '斯里兰卡',
    'Lesotho': '莱索托',
    'Lithuania': '立陶宛',
    'Luxembourg': '卢森堡',
    'Latvia': '拉脱维亚',
    'Morocco': '摩洛哥',
    'Moldova': '摩尔多瓦',
    'Madagascar': '马达加斯加',
    'Mexico': '墨西哥',
    'Macedonia': '马其顿',
    'Mali': '马里',
    'Myanmar': '缅甸',
    'Montenegro': '黑山',
    'Mongolia': '蒙古',
    'Mozambique': '莫桑比克',
    'Mauritania': '毛里塔尼亚',
    'Malawi': '马拉维',
    'Malaysia': '马来西亚',
    'Namibia': '纳米比亚',
    'New Caledonia': '新喀里多尼亚',
    'Niger': '尼日尔',
    'Nigeria': '尼日利亚',
    'Nicaragua': '尼加拉瓜',
    'Netherlands': '荷兰',
    'Norway': '挪威',
    'Nepal': '尼泊尔',
    'New Zealand': '新西兰',
    'Oman': '阿曼',
    'Pakistan': '巴基斯坦',
    'Panama': '巴拿马',
    'Peru': '秘鲁',
    'Philippines': '菲律宾',
    'Papua New Guinea': '巴布亚新几内亚',
    'Poland': '波兰',
    'Puerto Rico': '波多黎各',
    'Dem. Rep. Korea': '朝鲜',
    'Portugal': '葡萄牙',
    'Paraguay': '巴拉圭',
    'Qatar': '卡塔尔',
    'Romania': '罗马尼亚',
    'Russia': '俄罗斯',
    'Rwanda': '卢旺达',
    'W. Sahara': '西撒哈拉',
    'Saudi Arabia': '沙特阿拉伯',
    'Sudan': '苏丹',
    'S. Sudan': '南苏丹',
    'Senegal': '塞内加尔',
    'Solomon Is.': '所罗门群岛',
    'Sierra Leone': '塞拉利昂',
    'El Salvador': '萨尔瓦多',
    'Somaliland': '索马里兰',
    'Somalia': '索马里',
    'Serbia': '塞尔维亚',
    'Suriname': '苏里南',
    'Slovakia': '斯洛伐克',
    'Slovenia': '斯洛文尼亚',
    'Sweden': '瑞典',
    'Swaziland': '斯威士兰',
    'Syria': '叙利亚',
    'Chad': '乍得',
    'Togo': '多哥',
    'Thailand': '泰国',
    'Tajikistan': '塔吉克斯坦',
    'Turkmenistan': '土库曼斯坦',
    'East Timor': '东帝汶',
    'Trinidad and Tobago': '特里尼达和多巴哥',
    'Tunisia': '突尼斯',
    'Turkey': '土耳其',
    'Tanzania': '坦桑尼亚',
    'Uganda': '乌干达',
    'Ukraine': '乌克兰',
    'Uruguay': '乌拉圭',
    'United States': '美国',
    'Uzbekistan': '乌兹别克斯坦',
    'Venezuela': '委内瑞拉',
    'Vietnam': '越南',
    'Vanuatu': '瓦努阿图',
    'West Bank': '西岸',
    'Yemen': '也门',
    'South Africa': '南非',
    'Zambia': '赞比亚',
    'Zimbabwe': '津巴布韦',
    'Comoros': '科摩罗'
}

In [8]:
#2.2
city_data = pd.ExcelFile("附件1.xlsx",engine='openpyxl')
nation = city_data.parse('国际疫情')
tr = Timeline().add_schema(play_interval=1000, height=40, is_loop_play=True,
                           is_auto_play=True, control_position="left", is_rewind_play=False)
tl = Timeline().add_schema(play_interval=1000, height=40, is_loop_play=True, is_auto_play=True, control_position="left",
                           is_rewind_play=False)  # 时间轴
list1 = []  # 存放日期
list2 = []  # 存放确诊人数
list3 = []  # 存放治愈人数
list4 = []  # 存放死亡人数
for i in range(1, 7):  # 以月为单位进行遍历 得到1月到i月各个省份的累计确诊，治愈，死亡人数
    list1.append("2020-0{}".format(i))  # 存入日期
    data2 = nation
    data2['日期'] = pd.to_datetime(data2['日期'])  # 转换日期类型
    data2 = data2.set_index('日期')  # 将日期作为索引
    data2 = data2.sort_index()  # 对日期进行排序 以便于对其进行切分
    if i % 2 == 1:  # 1，3，5月
        data2 = data2.truncate('2020-01', '2020-0{}-31'.format(i))  # 对数据进行切分，得到1月到i月的全部数据
    else:
        if i == 2:  # 2月
            data2 = data2.truncate('2020-01', '2020-0{}-28'.format(i))  # 对数据进行切分，得到1月到i月的全部数据
        else:  # 4，6月
            data2 = data2.truncate('2020-01', '2020-0{}-30'.format(i))  # 对数据进行切分，得到1月到i月的全部数据
    data2 = data2.groupby('日期', as_index=False).sum()
    data2['日期'] = '2020-0{}'.format(i)  # 加入日期列
    list2.append(int(data2.groupby('日期', as_index=False).sum()['累计确诊']))  # 存入确诊人数
    list3.append(int(data2.groupby('日期', as_index=False).sum()['累计治愈']))  # 存入治愈人数
    list4.append(int(data2.groupby('日期', as_index=False).sum()['累计死亡']))  # 存入死亡人数

for i in range(1, 7):  # 以月为单位进行遍历
    data1 = nation
    data1['日期'] = pd.to_datetime(data1['日期'])  # 转换日期类型
    data1 = data1.set_index('日期')  # 将日期作为索引
    data1 = data1.sort_index()  # 对日期进行排序 以便于对其进行切分
    if i % 2 == 1:
        data1 = data1.truncate('2020-01', '2020-0{}-31'.format(i))
    else:
        if i == 2:
            data1 = data1.truncate('2020-01', '2020-0{}-28'.format(i))
        else:
            data1 = data1.truncate('2020-01', '2020-0{}-30'.format(i))
    data1 = data1.groupby('国家', as_index=False).max()  # 对1月到i月的各个省份的各项数据取最大值
    data1['日期'] = '2020-0{}'.format(i)  # 加入日期列
    data2 = data1
    data2 = data2.sort_values(by='累计确诊', ascending=False)  # 从大到小排序‘新增确诊’这一列，方便得到累计确诊人数前五的省份
    x = data2['国家'].tolist()  # 转换为list
    y = data2['累计确诊'].tolist()
    map = (  # 进行大屏显示 画图
        Map()
            .add(series_name="确诊病例",  # 系列名称，用于 tooltip 的显示，legend 的图例筛选。
                 data_pair=[list(z) for z in zip(x, y)],  # 数据项 (坐标点名称，坐标点值)
                 maptype='world',  # 地图类型
                 is_map_symbol_show=True,
                 name_map=name_map)
            .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(  # 全局配置项
            title_opts=opts.TitleOpts(title='疫情地图'),  # title_opts = opts.TitleOpts()主标题函数
            visualmap_opts=opts.VisualMapOpts(  # visualmap_opts=opts.VisualMapOpts()视觉映射函数
                is_piecewise=True,  # 是否分段
                # 自定义的每一段的范围，以及每一段的文字，以及每一段的特别的样式。
                pieces=[{"max": 9, "min": 0, "label": "0-9", "color": "#FFECEC"},
                        {"max": 99, "min": 10, "label": "10-99", "color": "	#FFB5B5"},
                        {"max": 499, "min": 100, "label": "100-4999", "color": "#ff7575"},
                        {"max": 999, "min": 500, "label": "500-999", "color": "#FF2D2D"},
                        {"max": 9999, "min": 1000, "label": "1000-9999", "color": "	#EA0000"},
                        {"max": 99999, "min": 10000, "label": "10000-99999", "color": "#AE0000"},
                        {"max": 999999, "min": 100000, "label": "100000-999999", "color": "	#750000"},
                        {"max": 99999999999, "min": 100000, "label": ">999999", "color": "#4D0000"}]
            )
        )
    )

    pie = (  # 画饼图
        Pie()

            .add(
            series_name="累计确诊",
            radius=["40%", "55%"],
            data_pair=[list(z) for z in zip(x[:3], y[:3])],  # 前3个省份
            label_opts=opts.LabelOpts(
                position="outside",
                formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
                background_color="#eee",
                border_color="#aaa",
                border_width=1,
                border_radius=4,
                rich={
                    "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                    "abg": {
                        "backgroundColor": "#e3e3e3",
                        "width": "100%",
                        "align": "right",
                        "height": 22,
                        "borderRadius": [4, 4, 0, 0],
                    },
                    "hr": {
                        "borderColor": "#aaa",
                        "width": "100%",
                        "borderWidth": 0.5,
                        "height": 0,
                    },
                    "b": {"fontSize": 16, "lineHeight": 33},
                    "per": {
                        "color": "#eee",
                        "backgroundColor": "#334455",
                        "padding": [2, 4],
                        "borderRadius": 2,
                    },
                },
            )
        )
            .set_global_opts(legend_opts=opts.LegendOpts(pos_left="left", orient="vertical"))
            .set_series_opts(
            tooltip_opts=opts.TooltipOpts(
                trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
            )
        )
    )
    line = (  # 画折线图
        Line()
            .add_xaxis(xaxis_data=list1)
            .add_yaxis("累计确诊人数", y_axis=list2, is_smooth=True)
            .add_yaxis("累计治愈人数", y_axis=list3, is_smooth=True)
            .add_yaxis("累计确诊死亡", y_axis=list4, is_smooth=True)
            .set_series_opts(
            areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
            .set_global_opts(
            title_opts=opts.TitleOpts(title="全球疫情"),
            tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
            yaxis_opts=opts.AxisOpts(
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
            xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        )
    )

    tl.add(pie, "2020-0{}".format(i))  # 将不同日期的数据加入到timeline中
    tr.add(map, "2020-0{}".format(i))  # 将不同日期的数据加入到timeline中
page = Page(layout=Page.DraggablePageLayout)  # 合并在一页
page.add(tr, tl, line)
page.render('task2_2.html')  # 将结果保存

C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2945394982.py:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data2 = data2.groupby('日期', as_index=False).sum()
C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2945394982.py:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data2 = data2.groupby('日期', as_index=False).sum()
C:\Users\吕强\AppData\Local\Temp\ipykernel_18332\2945394982.py:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the fun

'c:\\Users\\吕强\\Desktop\\week3\\task2_2.html'